* 문서간 동일한 단어 또는 비숫한 단어가 공통적으로 들어 있는지를 수치화
* 문서간 단의 차이를 어떤 방법으로 계산하는지에 따라 방법이 다양할 수 있음

### 1. 코사인 유사도
* 두 벡터간 코사인 각도를 이용하여 두벡터의 유사도를 구함
* 두 벡터가 방향이 동일하면 유사도는 1을 갖게 되고, 반대방향(-180도)이면 -1 값을 갖게 됨

$$similarity=cos(Θ)=\frac{A⋅B}{||A||\ ||B||}=\frac{\sum_{i=1}^{n}{A_{i}×B_{i}}}{\sqrt{\sum_{i=1}^{n}(A_{i})^2}×\sqrt{\sum_{i=1}^{n}(B_{i})^2}}$$

* 문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우, 각각의 특징벡터 A, B가 만들어짐.

In [3]:
### 시뮬레이션
def cos_sim(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [4]:
doc1 = np.array([0, 1, 1, 1])  # 저는 사과 좋아요
doc2 = np.array([1, 0, 1, 1])  # 저는 바나나 좋아요
doc3 = np.array([2, 0, 2, 2])  # 저는 바나나 좋아요 저는 바나나 좋아요

print('문서 1과 문서 2의 유사도: ', cos_sim(doc1, doc2))
print('문서 1과 문서 3의 유사도: ', cos_sim(doc1, doc3))
print('문서 2과 문서 3의 유사도: ', cos_sim(doc2, doc3))

문서 1과 문서 2의 유사도:  0.6666666666666667
문서 1과 문서 3의 유사도:  0.6666666666666667
문서 2과 문서 3의 유사도:  1.0000000000000002


* 주목할 점은 문서1과 문서3의 유사도와 문서1과 문서 3과의 유사도가 같다는 점
* 그리고 문서2와 문서3은 유사도 1이라는 점... 문서의 길이가 차이가 나도 유사도 계산이 가능하다는 점을 시사함. 왜냐하면 벡터의 방향에 초점을 두고 있기 때문임
* 따라서 코사인 유사도는 비교적 다른 상황에서 공정한 비교가 가능함

### 2. 유사도를 이용한 추천시스템 구현하기

* kaggle dataset : https://www.kaggle.com/rounakbanik/the-movies-dataset
* movies_metadata.csv : 24개 컬럼을 가진 25,466개 샘플 영화정보 데이터

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.tail(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [7]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

#### title과 overview 사용하여 유사한 줄거리의 영화를 찾아서 추천하는 시스템 만들기

In [8]:
### 상위 20000개만 data에 저장
data = data[:20000]

In [10]:
data.tail(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19998,False,NaN,0,[],NaN,197723,tt0471239,en,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...,...,2001-11-19,0.0,0.0,[],Released,NaN,Lotte Reiniger: Homage to the Inventor of the ...,False,0.0,0.0
19999,False,NaN,0,[],NaN,269797,tt0492337,en,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a...",...,2005-10-23,0.0,0.0,[],Released,NaN,"RKO Production 601: The Making of 'Kong, the E...",False,8.0,1.0


In [11]:
### 결측값 처리
print('overview 열의 결측값 수:', data.overview.isnull().sum())

overview 열의 결측값 수: 135


In [12]:
data.overview = data.overview.fillna('')  # Null 값을 빈값으로 대체

In [13]:
print('overview 열의 결측값 수:', data.overview.isnull().sum())

overview 열의 결측값 수: 0


In [14]:
### TF-IDF 행렬 구하기

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data.overview)

In [15]:
print('TF-IDF 행렬 크기:', tfidf_matrix.shape)  ## 47,487개 단어가 사용됨

TF-IDF 행렬 크기: (20000, 47487)


In [18]:
tfidf_matrix[0].toarray().shape

(1, 47487)

In [21]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산결과: ', cosine_sim.shape)  # 2만개 문서간 비교이기 때문에 행렬크기가 이렇게 바뀜

코사인 유사도 연산결과:  (20000, 20000)


#### 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index 만들기

In [22]:
title_to_index = dict(zip(data['title'], data.index))

In [23]:
zip(data['title'], data.index)

In [24]:
data.index

RangeIndex(start=0, stop=20000, step=1)

In [26]:
title_to_index['Father of the Bride Part II']

4

In [27]:
cosine_sim[4]

array([0.        , 0.        , 0.02500492, ..., 0.        , 0.        ,
       0.        ])

In [43]:
sim_scores = list(enumerate(cosine_sim[4]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
for idx, sim in sim_scores[1:11]:
    print(data['title'].iloc[idx])

Father of the Bride
Kuffs
North to Alaska
Babbitt
Wendigo
The Magic of Méliès
The Out of Towners
It's a Wonderful Life
Funny People
All Night Long


In [45]:
### 영화 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title]
    
    # 해당 영화와 모든 영화와의 유사도 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도가 높은 순으로 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # 가장 유사한 10개의 영화 유사도 받아오기
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개영화 인덱스 얻기
    movie_indices = [idx[0] for idx in sim_scores]
    
    return data['title'].iloc[movie_indices]

In [46]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [53]:
### 유사도가 0.1보다 큰 영화는 90개

count = 0
for idx, sim in sim_scores:
    if sim > 0.05:
        count += 1
        
count

404

In [49]:
len(sim_scores)

20000